# Verification of New Results for Bots Case (revised)
#### Unified Model with Integrated Bot 

James Yu, 14 January 2023

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
np.set_printoptions(suppress=True) # disable scientific notation for readability

In [2]:
def infinite_solution(A_n, A_c, delta, c, x_0, z, b = np.identity(3)):
    n = len(x_0)
    eps = np.finfo(np.float64).eps
    I = np.identity(n)
    O = np.zeros((n, n))
    
    Q = np.block([
        [I, O], 
        [O, I]
    ])
    A = np.block([
        [A_n, A_c], 
        [O, I]
    ])
    B = np.block([
        [b], # identity matrix by default 
        [O]
    ])
    K_t = Q
    K_sequence = [K_t]
    
    while True: # generate solution matrices; note I divide delta into the inverse term for simplification
        K_t_new = Q + delta * A.T @ (K_t - K_t @ B @ inv(B.T @ K_t @ B + c * I / delta) @ B.T @ K_t) @ A
        K_sequence.insert(0, K_t_new)
        if np.allclose(K_t, K_t_new, rtol = eps, atol = eps): break
        K_t = K_t_new

    chi_var = np.block([
        [x_0], 
        [z]
    ])
    chi_ts = [chi_var]
    r_ts = []
    K_ss = K_sequence[0]
    L_ss = -inv(B.T @ K_ss @ B + c * I / delta) @ B.T @ K_ss @ A
    
    payoff = 0
    stage_payoffs = []
    discounted_stage_payoffs = []
    cumulative_payoffs = []
    
    i = 0
    while True:
        r_t = L_ss @ chi_var
        r_ts.append(r_t)
        p = -(chi_var.T @ Q @ chi_var + c * r_t.T @ r_t).item()
        payoff += delta**i * p
        stage_payoffs.append(p)
        discounted_stage_payoffs.append(delta**i * p)
        cumulative_payoffs.append(payoff)
        chi_var_new = A @ chi_var + B @ r_t
        chi_ts.append(chi_var_new)
        if np.allclose(chi_var, chi_var_new, rtol = eps, atol = eps): break
        chi_var = chi_var_new
        i += 1
        
    return chi_ts, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss

# 0. Setup

In [3]:
A_1_n = 0.99 * np.array([
    [0.3, 0.4, 0.3],
    [0.6, 0.2, 0.2],
    [0.5, 0.1, 0.4]
])
A_1_c = np.diag([0.01, 0.01, 0.01])

delta = 0.9
c = 2.0
x_0 = np.array([[10.0, 5.0, -20.0]], ndmin = 2).T
z = np.array([[5.0, 5.0, 5.0]], ndmin = 2).T

In [4]:
chi_ts, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss = infinite_solution(A_1_n, A_1_c, delta, c, x_0, z)

In [5]:
print(K_ss)

[[ 1.59050852  0.27408037  0.3645815   0.00776106  0.00776879  0.00759601]
 [ 0.27408037  1.18754682  0.1911302   0.00581348  0.0038073   0.00330085]
 [ 0.3645815   0.1911302   1.25807934  0.00584039  0.00408362  0.00577534]
 [ 0.00776106  0.00581348  0.00584039 10.00125739  0.00053347  0.00052138]
 [ 0.00776879  0.0038073   0.00408362  0.00053347 10.00085423  0.00019268]
 [ 0.00759601  0.00330085  0.00577534  0.00052138  0.00019268 10.00105938]]


In [6]:
print(L_ss)

[[-0.17701011 -0.17633273 -0.15360112 -0.00587717 -0.00232565 -0.0023977 ]
 [-0.23242086 -0.08853252 -0.09349508 -0.00192399 -0.0043429  -0.00105756]
 [-0.21136216 -0.06481019 -0.16390937 -0.00200436 -0.0012404  -0.00496504]]


In [7]:
chi_ts[-1] # terminal opinions

array([[0.00330152],
       [0.01796757],
       [0.01367575],
       [5.        ],
       [5.        ],
       [5.        ]])

In [8]:
r_ts[-1] # terminal messages

array([[-0.05885589],
       [-0.04025891],
       [-0.04515289]])

# 1. Expression for $K_1$ component of the block matrix $K^*$

Recall that this expression was:

$$
K_1 = I + c\delta(A^n)'K_1(\delta K_1 + cI)^{-1}A^n
$$

In [9]:
print(K_ss)

[[ 1.59050852  0.27408037  0.3645815   0.00776106  0.00776879  0.00759601]
 [ 0.27408037  1.18754682  0.1911302   0.00581348  0.0038073   0.00330085]
 [ 0.3645815   0.1911302   1.25807934  0.00584039  0.00408362  0.00577534]
 [ 0.00776106  0.00581348  0.00584039 10.00125739  0.00053347  0.00052138]
 [ 0.00776879  0.0038073   0.00408362  0.00053347 10.00085423  0.00019268]
 [ 0.00759601  0.00330085  0.00577534  0.00052138  0.00019268 10.00105938]]


In [10]:
print(K_ss[0:3,0:3]) # K_1

[[1.59050852 0.27408037 0.3645815 ]
 [0.27408037 1.18754682 0.1911302 ]
 [0.3645815  0.1911302  1.25807934]]


In [11]:
K_1 = K_ss[0:3,0:3]
LHS = np.identity(3) + c*delta*A_1_n.T @ K_1 @ inv(delta*K_1 + c*np.identity(3)) @ A_1_n
np.allclose(LHS, K_1) # is approximately equal?

True

In [12]:
LHS

array([[1.59050852, 0.27408037, 0.3645815 ],
       [0.27408037, 1.18754682, 0.1911302 ],
       [0.3645815 , 0.1911302 , 1.25807934]])

They appear to be the same. To decompose:

In [13]:
c*delta*A_1_n.T @ K_1

array([[1.4681747 , 1.58654543, 1.52021037],
       [1.29636514, 0.65266558, 0.55218223],
       [1.20784179, 0.70600266, 1.15978302]])

In [14]:
inv(delta*K_1 + c*np.identity(3)) 

array([[ 0.29585837, -0.02211217, -0.02977851],
       [-0.02211217,  0.32851996, -0.01572519],
       [-0.02977851, -0.01572519,  0.3232402 ]])

In [15]:
c*delta*A_1_n.T @ K_1 @ inv(delta*K_1 + c*np.identity(3)) @ A_1_n

array([[0.59050852, 0.27408037, 0.3645815 ],
       [0.27408037, 0.18754682, 0.1911302 ],
       [0.3645815 , 0.1911302 , 0.25807934]])

# 2. Expression for $K_2$ component of the block matrix $K^*$

$$
K_2 = c \delta {A^n}' (\delta K_1 + cI)^{-1} (K_1 A^z + K_2)
$$

In [16]:
print(K_ss)

[[ 1.59050852  0.27408037  0.3645815   0.00776106  0.00776879  0.00759601]
 [ 0.27408037  1.18754682  0.1911302   0.00581348  0.0038073   0.00330085]
 [ 0.3645815   0.1911302   1.25807934  0.00584039  0.00408362  0.00577534]
 [ 0.00776106  0.00581348  0.00584039 10.00125739  0.00053347  0.00052138]
 [ 0.00776879  0.0038073   0.00408362  0.00053347 10.00085423  0.00019268]
 [ 0.00759601  0.00330085  0.00577534  0.00052138  0.00019268 10.00105938]]


In [17]:
print(K_ss[0:3, 3:6]) # K_2

[[0.00776106 0.00776879 0.00759601]
 [0.00581348 0.0038073  0.00330085]
 [0.00584039 0.00408362 0.00577534]]


In [18]:
K_2 = K_ss[0:3, 3:6]
LHS_2 = c * delta * A_1_n.T @ inv(c * np.identity(3) + delta * K_1) @ (K_2 + K_1 @ A_1_c)
np.allclose(LHS_2, K_2) # is approximately equal?

True

In [19]:
LHS_2

array([[0.00776106, 0.00776879, 0.00759601],
       [0.00581348, 0.0038073 , 0.00330085],
       [0.00584039, 0.00408362, 0.00577534]])

# 3. Expression for $x_{\infty}$ terminal opinions

$$
x_\infty = (\delta K_1 + cI - cA^n)^{-1}(cA^z - \delta K_2)z
$$

In [20]:
LHS_3 = inv(delta * K_1 + c*np.identity(3) - c * A_1_n) @ (c * A_1_c - delta * K_2) @ z
LHS_3

array([[0.00330152],
       [0.01796757],
       [0.01367575]])

In [21]:
chi_ts[-1][0:3]

array([[0.00330152],
       [0.01796757],
       [0.01367575]])

In [22]:
np.allclose(chi_ts[-1][0:3], LHS_3)

True

Checking limit vanishing:

In [23]:
(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n)

array([[0.11998989, 0.21966727, 0.14339888],
       [0.36157914, 0.10946748, 0.10450492],
       [0.28363784, 0.03418981, 0.23209063]])

In [24]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 2)

array([[0.13449802, 0.05530705, 0.07344426],
       [0.11260855, 0.09498323, 0.08754455],
       [0.11222568, 0.07398375, 0.09811241]])

In [25]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 3)

array([[0.05696785, 0.03811018, 0.04211245],
       [0.07268679, 0.03812712, 0.04639242],
       [0.06804532, 0.03610557, 0.04659567]])

In [26]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 100)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])